In [26]:
import sqlite3
import requests
import json


# DBファイルを保存するファイルパス
# Google Colab
path = 'jma'

# DBファイル名
db_name = 'Dsprog2.db'
# db_name = 'dsprg2.sqlite'

# DBに接続する（DBファイルが存在しない場合は，新規に作成される）
con = sqlite3.connect(path + db_name)


# DBへの接続を閉じる
con.close()

In [27]:
# DBに接続
con = sqlite3.connect(path + db_name)

# SQL（RDBを操作するための言語）を実行するためのカーソルオブジェクトを取得
cur = con.cursor()

# 実行したいSQL
# 一旦
sql = """CREATE TABLE IF NOT EXISTS offices (
    id TEXT PRIMARY KEY,
    name TEXT,
    en_name TEXT,
    weatherCodes TEXT,
    timeSeries TEXT

)"""

# SQLを実行
cur.execute(sql)

# DBへの接続を閉じる
con.close()

In [28]:
# 地域コードのリスト（idのリスト）
office_ids = [
    "011000", "012000", "013000", "014030", "014100", 
    "015000", "016000", "017000", "020000", "030000", 
    "040000", "050000", "060000", "070000", "080000", 
    "090000", "100000", "110000", "120000", "130000", 
    "140000", "150000", "160000", "170000", "180000", 
    "190000", "200000", "210000", "220000", "230000", 
    "240000", "250000", "260000", "270000", "280000", 
    "290000", "300000", "310000", "320000", "330000", 
    "340000", "350000", "360000", "370000", "380000", 
    "390000", "400000", "410000", "420000", "430000", 
    "440000", "450000", "460040", "460100", "471000", 
    "472000", "473000", "474000"
]

# API URLのフォーマット
base_url = "https://www.jma.go.jp/bosai/forecast/data/forecast/{}.json"

# 取得したデータを格納するリスト
all_office_data = []

for office_id in office_ids:
    url = base_url.format(office_id)
    try:
        # APIリクエスト
        response = requests.get(url, timeout=10)  # 10秒でタイムアウト
        response.raise_for_status()
        
        # JSONデータを取得
        weather_data = response.json()
        
        # 必要なデータを取得
        if weather_data:
            time_defines = weather_data[0].get("timeSeries", [])[0].get("timeDefines", [])
            weather_codes = weather_data[0].get("timeSeries", [])[0].get("areas", [])[0].get("weatherCodes", [])
            
            office_info = {
                "id": office_id,
                "timeDefines": time_defines,
                "weatherCodes": weather_codes
            }
            all_office_data.append(office_info)
            
            print(f"{office_id}: データ取得成功")
        else:
            print(f"{office_id}: データが空でした")
    
    except requests.exceptions.RequestException as e:
        print(f"{office_id}: リクエストエラーが発生しました - {e}")
    except Exception as e:
        print(f"{office_id}: 想定外のエラーが発生しました - {e}")

# 取得したデータをファイルに保存（オプション）
with open('weather_data.json', 'w', encoding='utf-8') as file:
    json.dump(all_office_data, file, ensure_ascii=False, indent=4)



011000: データ取得成功
012000: データ取得成功
013000: データ取得成功
014030: リクエストエラーが発生しました - 404 Client Error: Not Found for url: https://www.jma.go.jp/bosai/forecast/data/forecast/014030.json
014100: データ取得成功
015000: データ取得成功
016000: データ取得成功
017000: データ取得成功
020000: データ取得成功
030000: データ取得成功
040000: データ取得成功
050000: データ取得成功
060000: データ取得成功
070000: データ取得成功
080000: データ取得成功
090000: データ取得成功
100000: データ取得成功
110000: データ取得成功
120000: データ取得成功
130000: データ取得成功
140000: データ取得成功
150000: データ取得成功
160000: データ取得成功
170000: データ取得成功
180000: データ取得成功
190000: データ取得成功
200000: データ取得成功
210000: データ取得成功
220000: データ取得成功
230000: データ取得成功
240000: データ取得成功
250000: データ取得成功
260000: データ取得成功
270000: データ取得成功
280000: データ取得成功
290000: データ取得成功
300000: データ取得成功
310000: データ取得成功
320000: データ取得成功
330000: データ取得成功
340000: データ取得成功
350000: データ取得成功
360000: データ取得成功
370000: データ取得成功
380000: データ取得成功
390000: データ取得成功
400000: データ取得成功
410000: データ取得成功
420000: データ取得成功
430000: データ取得成功
440000: データ取得成功
450000: データ取得成功
460040: リクエストエラーが発生しました - 404 Client Error: Not Found for 

In [29]:
import json

# weather_info.json の読み込み
with open("weather_info.json", "r", encoding="utf-8") as file:
    data_offices = json.load(file)

# weather_data.json の読み込み
with open("weather_data.json", "r", encoding="utf-8") as file:
    data_weather = json.load(file)

# offices キーのデータから ID と地名を抽出
office_info = []
for office_id, office_data in data_offices["offices"].items():
    office_info.append({
        "ID": office_id,
        "Name": office_data["name"],
        "EnName": office_data["enName"]
    })

# weather_data.json から timeDefines と weatherCodes を統合
final_data = []
for office in office_info:
    for weather in data_weather:
        if office["ID"] == weather["id"]:  # IDが一致する場合に統合
            final_data.append({
                "ID": office["ID"],
                "Name": office["Name"],
                "EnName": office["EnName"],
                "timeDefines": weather["timeDefines"],
                "weatherCodes": weather["weatherCodes"]
            })

# 統合したデータを表示（上位5件）
print("ID\tName\t\t\tEnName\t\t\tTimeDefines\t\tWeatherCodes")
print("-" * 100)
for row in final_data:
    # 表示データをシングルクォートで囲んで出力
    time_defines = ', '.join([f"'{t}'" for t in row['timeDefines']])
    weather_codes = ', '.join([f"'{w}'" for w in row['weatherCodes']])
    
    print(f"('{row['ID']}', '{row['Name']}', '{row['EnName']}', json.dumps([{weather_codes}]), json.dumps([{time_defines}])),")

ID	Name			EnName			TimeDefines		WeatherCodes
----------------------------------------------------------------------------------------------------
('011000', '宗谷地方', 'Soya', json.dumps(['205', '205', '205']), json.dumps(['2024-12-17T17:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('012000', '上川・留萌地方', 'Kamikawa Rumoi', json.dumps(['217', '205', '205']), json.dumps(['2024-12-17T17:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('013000', '網走・北見・紋別地方', 'Abashiri Kitami Mombetsu', json.dumps(['111', '110', '201']), json.dumps(['2024-12-17T17:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('014100', '釧路・根室地方', 'Kushiro Nemuro', json.dumps(['101', '101', '101']), json.dumps(['2024-12-17T17:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('015000', '胆振・日高地方', 'Iburi Hidaka', json.dumps(['201', '201', '201']), json.dumps(['2024-12-17T17:00:00+09:00', '2024-12-18T00:00:00+09:00', '20

In [30]:
con = sqlite3.connect(path + db_name)

# SQL（RDBを操作するための言語）を実行するためのカーソルオブジェクトを取得
cur = con.cursor()

# データを挿入するSQL
sql_insert_many = "INSERT INTO offices VALUES (?, ?, ?, ?, ?);"


# データを挿入

offices = [
('011000', '宗谷地方', 'Soya', json.dumps(['400', '205', '205']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('012000', '上川・留萌地方', 'Kamikawa Rumoi', json.dumps(['205', '205', '205']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('013000', '網走・北見・紋別地方', 'Abashiri Kitami Mombetsu', json.dumps(['101', '201', '201']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('014100', '釧路・根室地方', 'Kushiro Nemuro', json.dumps(['100', '101', '101']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('015000', '胆振・日高地方', 'Iburi Hidaka', json.dumps(['201', '201', '201']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('016000', '石狩・空知・後志地方', 'Ishikari Sorachi Shiribeshi', json.dumps(['104', '205', '204']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('017000', '渡島・檜山地方', 'Oshima Hiyama', json.dumps(['200', '204', '200']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('020000', '青森県', 'Aomori', json.dumps(['217', '402', '205']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('030000', '岩手県', 'Iwate', json.dumps(['201', '210', '201']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('040000', '宮城県', 'Miyagi', json.dumps(['111', '101', '101']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('050000', '秋田県', 'Akita', json.dumps(['400', '402', '205']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('060000', '山形県', 'Yamagata', json.dumps(['402', '413', '205']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('070000', '福島県', 'Fukushima', json.dumps(['111', '210', '101']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('080000', '茨城県', 'Ibaraki', json.dumps(['101', '111', '101']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('090000', '栃木県', 'Tochigi', json.dumps(['101', '110', '101']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('100000', '群馬県', 'Gunma', json.dumps(['101', '101', '101']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('110000', '埼玉県', 'Saitama', json.dumps(['100', '111', '101']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('120000', '千葉県', 'Chiba', json.dumps(['100', '110', '101']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('130000', '東京都', 'Tokyo', json.dumps(['100', '111', '101']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('140000', '神奈川県', 'Kanagawa', json.dumps(['100', '110', '101']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('150000', '新潟県', 'Niigata', json.dumps(['304', '402', '270']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('160000', '富山県', 'Toyama', json.dumps(['302', '302', '270']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('170000', '石川県', 'Ishikawa', json.dumps(['302', '304', '270']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('180000', '福井県', 'Fukui', json.dumps(['302', '304', '270']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('190000', '山梨県', 'Yamanashi', json.dumps(['100', '111', '101']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('200000', '長野県', 'Nagano', json.dumps(['201', '205', '204']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('210000', '岐阜県', 'Gifu', json.dumps(['101', '200', '101']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('220000', '静岡県', 'Shizuoka', json.dumps(['100', '100', '101']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('230000', '愛知県', 'Aichi', json.dumps(['100', '101', '101']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('240000', '三重県', 'Mie', json.dumps(['101', '101', '101']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('250000', '滋賀県', 'Shiga', json.dumps(['101', '201', '200']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('260000', '京都府', 'Kyoto', json.dumps(['101', '201', '260']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('270000', '大阪府', 'Osaka', json.dumps(['201', '201', '201']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('280000', '兵庫県', 'Hyogo', json.dumps(['101', '201', '260']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('290000', '奈良県', 'Nara', json.dumps(['101', '201', '201']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('300000', '和歌山県', 'Wakayama', json.dumps(['101', '201', '201']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('310000', '鳥取県', 'Tottori', json.dumps(['200', '281', '371']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('320000', '島根県', 'Shimane', json.dumps(['200', '281', '371']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('330000', '岡山県', 'Okayama', json.dumps(['101', '101', '201']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('340000', '広島県', 'Hiroshima', json.dumps(['201', '201', '201']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('350000', '山口県', 'Yamaguchi', json.dumps(['200', '200', '206']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('360000', '徳島県', 'Tokushima', json.dumps(['200', '200', '260']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('370000', '香川県', 'Kagawa', json.dumps(['201', '200', '206']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('380000', '愛媛県', 'Ehime', json.dumps(['201', '200', '206']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('390000', '高知県', 'Kochi', json.dumps(['101', '201', '201']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('400000', '福岡県', 'Fukuoka', json.dumps(['200', '200', '206']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('410000', '佐賀県', 'Saga', json.dumps(['111', '200', '206']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('420000', '長崎県', 'Nagasaki', json.dumps(['200', '200', '201']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('430000', '熊本県', 'Kumamoto', json.dumps(['200', '200', '201']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('440000', '大分県', 'Oita', json.dumps(['101', '210', '200']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('450000', '宮崎県', 'Miyazaki', json.dumps(['211', '111', '100']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('460100', '鹿児島県（奄美地方除く）', 'Kagoshima (Excluding Amami)', json.dumps(['201', '200', '101']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('471000', '沖縄本島地方', 'Okinawa Main Island', json.dumps(['100', '111', '200']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('472000', '大東島地方', 'Daitojima', json.dumps(['211', '110', '201']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('473000', '宮古島地方', 'Miyakojima', json.dumps(['101', '111', '200']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),
('474000', '八重山地方', 'Yaeyama', json.dumps(['111', '200', '200']), json.dumps(['2024-12-17T11:00:00+09:00', '2024-12-18T00:00:00+09:00', '2024-12-19T00:00:00+09:00'])),

]

# SQLを実行
cur.executemany(sql_insert_many, offices)

# コミット処理（データ操作を反映させる）
con.commit()

# DBへの接続を閉じる
con.close()


In [31]:
# DBに接続
con = sqlite3.connect(path + db_name)

# SQL（RDBを操作するための言語）を実行するためのカーソルオブジェクトを取得
cur = con.cursor()

# データを参照するSQL
# SELECT * FROM テーブル名;
# * の部分は，取得したい列の名前を，区切りで指定することもできる．
sql_select = "SELECT * FROM offices;"

# SQLを実行
cur.execute(sql_select)

for r in cur:
  # 行データ(r)はタプルなので，アンパックして列データを取得
  id,name, en_name, weathercodes, timeSeries = r
  print(id,name, en_name, weathercodes, timeSeries)
  # print(r)

# DBへの接続を閉じる
con.close()

011000 宗谷地方 Soya ["400", "205", "205"] ["2024-12-17T11:00:00+09:00", "2024-12-18T00:00:00+09:00", "2024-12-19T00:00:00+09:00"]
012000 上川・留萌地方 Kamikawa Rumoi ["205", "205", "205"] ["2024-12-17T11:00:00+09:00", "2024-12-18T00:00:00+09:00", "2024-12-19T00:00:00+09:00"]
013000 網走・北見・紋別地方 Abashiri Kitami Mombetsu ["101", "201", "201"] ["2024-12-17T11:00:00+09:00", "2024-12-18T00:00:00+09:00", "2024-12-19T00:00:00+09:00"]
014100 釧路・根室地方 Kushiro Nemuro ["100", "101", "101"] ["2024-12-17T11:00:00+09:00", "2024-12-18T00:00:00+09:00", "2024-12-19T00:00:00+09:00"]
015000 胆振・日高地方 Iburi Hidaka ["201", "201", "201"] ["2024-12-17T11:00:00+09:00", "2024-12-18T00:00:00+09:00", "2024-12-19T00:00:00+09:00"]
016000 石狩・空知・後志地方 Ishikari Sorachi Shiribeshi ["104", "205", "204"] ["2024-12-17T11:00:00+09:00", "2024-12-18T00:00:00+09:00", "2024-12-19T00:00:00+09:00"]
017000 渡島・檜山地方 Oshima Hiyama ["200", "204", "200"] ["2024-12-17T11:00:00+09:00", "2024-12-18T00:00:00+09:00", "2024-12-19T00:00:00+09:00"]
020000 